In [ ]:
# ============================================================
# CIERRE FINAL DE DE (PSEUDOBULK) COMO SUPLEMENTARIA
# - Verifica que existen los volcanos FOR_FIGURE (Mono y T)
# - Ensambla SuppFigS1 (A=Mono, B=T) en un solo PNG
# - Copia todo a final_deliverables/
# ============================================================

from pathlib import Path
import shutil
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

NOTEBOOK_DIR = Path.cwd()

def find_project_root(start: Path) -> Path:
    for p in [start] + list(start.parents):
        if (p / "data_processed").exists():
            return p
    raise FileNotFoundError(f"No encuentro 'data_processed' subiendo desde: {start}")

PROJECT_ROOT = find_project_root(NOTEBOOK_DIR)
FIG_DIR = PROJECT_ROOT / "figures_final"
SUM_DIR = PROJECT_ROOT / "summary_tables_final"
OUT_DIR = PROJECT_ROOT / "final_deliverables"
OUT_DIR.mkdir(exist_ok=True)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("FIG_DIR     :", FIG_DIR)
print("SUM_DIR     :", SUM_DIR)
print("OUT_DIR     :", OUT_DIR)

# --- archivos esperados (ajusta SOLO si cambiaste nombres) ---
pMono = FIG_DIR / "Volcano_pseudobulk_Mono_Healthy_vs_Cirrhosis_FOR_FIGURE.png"
pT    = FIG_DIR / "Volcano_pseudobulk_T_Healthy_vs_Cirrhosis_FOR_FIGURE.png"

need = [pMono, pT]
missing = [str(p) for p in need if not p.exists()]
if missing:
    raise FileNotFoundError("Faltan volcanos FOR_FIGURE:\n- " + "\n- ".join(missing))

# --- ensamblar SuppFigS1 ---
imgM = mpimg.imread(pMono)
imgT = mpimg.imread(pT)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
for ax, img, lab, ttl in [
    (axes[0], imgM, "A", "Mono: Healthy vs Cirrhosis (pseudobulk, por paciente)"),
    (axes[1], imgT, "B", "T: Healthy vs Cirrhosis (pseudobulk, por paciente)"),
]:
    ax.imshow(img)
    ax.axis("off")
    ax.text(0.01, 0.99, lab, transform=ax.transAxes, va="top", ha="left",
            fontsize=14, fontweight="bold")
    ax.set_title(ttl, fontsize=10)

plt.tight_layout()
out_supp = FIG_DIR / "SuppFigS1_Volcanos_pseudobulk_Mono_T.png"
fig.savefig(out_supp, dpi=300, bbox_inches="tight")
plt.close(fig)
print("Saved:", out_supp)

# --- copiar al paquete final ---
to_copy = {
    "SuppFigS1_Volcanos_pseudobulk_Mono_T.png": out_supp,
    "SuppFigS1A_Volcano_pseudobulk_Mono.png": pMono,
    "SuppFigS1B_Volcano_pseudobulk_T.png": pT,

    # tablas soporte típicas que ya estás generando
    "TableS1_DE_pseudobulk_top10_by_celltype_reporting.csv": SUM_DIR / "DE_pseudobulk_FOR_REPORT_LINFO_CLEAN_top10_by_celltype.csv",
    "TableS2_scCODA_Level1refined_credible_effects_ref-T.csv": SUM_DIR / "scCODA_Level1refined_credible_effects_ref-T.csv",
}

print("\nCopiando a final_deliverables/ ...")
for name, src in to_copy.items():
    if not src.exists():
        print("[SKIP missing]", name, "->", src)
        continue
    dst = OUT_DIR / name
    shutil.copy2(src, dst)
    print("[OK]", dst)

print("\n[OK] DE (pseudobulk) cerrado como suplementaria.")
print("Contenido final_deliverables (resumen):")
for p in sorted(OUT_DIR.glob("*")):
    print(" -", p.name)
